<a href="https://colab.research.google.com/github/Fishing-oboro/NLP-learn/blob/main/index/Doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doc2vec (Paragraph Vector)
---
カウントベースの手法によって、文章ベクトルの獲得が可能である。しかし、その手法は単語の語順や意味を表現することができないという欠点を持つ。Doc2vecでは、その欠点を克服するためにword2vecの手法を文章ベクトルの獲得に取り入れている。  
これには、PV-DBOWとPV-DMという二つの手法があり、word2vecで用いられる2つの手法に類似している。
## Doc2vecの特徴
---
1. 可変長の文章から固定長の文章ベクトルを得ることができる。
2. word2vecと同様に単語を予測する過程でベクトルを得る。
3. word2vecとほぼ同一のニューラルネットワークを用いて学習する。



## PV-DM(分散記憶モデル)
---
この手法は、**CBOWと同様に周辺単語(コンテキスト)から中心単語(ターゲット)**を推測することを目的としたニューラルネットワークを活用する。入力層を**周辺単語ベクトル＋文章ベクトル**とすることで文章全体を加味したベクトルを得ることができる。

#### NNの構造
---
![](https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/image/PV%E3%83%BCDM.jpg)
1. 入力層:コンテキストに重みWin、文章IDに重みDinを使って低次元のベクトルに変換
2. 中間層:複数のベクトルを結合(和、平均など)
3. 出力層:重みを使って元の次元のベクトルに変換し、softMax関数で確率化。
4. 誤差逆伝搬:正解ラベルと確率の誤差が小さくなるように重みDinを更新。


#### PV-DMの特徴
---
周辺単語をベクトル化する際にWinを必要とするため、あらかじめ文章中の全単語の単語ベクトルを手に入れておく必要がある。また、文章ベクトルはDinから得ることができ、word2vecにおけるWinと同様のものである。

## PV-DBOW(分散BOWモデル)
--- 

この手法は、**文書IDから文中の単語(ターゲット)**を推測することを目的としたニューラルネットワークを活用する。入力層が文章IDのみであるため、**単語ベクトルを必要としない**。  
しかし、語順を無視して文中の単語を予測するため、文脈を加味しないベクトルになる。
#### NNの構造 
---

![](https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/image/PV-DBOW.jpg)

1. 入力層:文章IDに重みDinを使って低次元のベクトルに変換
2. 中間層:ベクトルを分割
3. 出力層:重みを使って元の次元のベクトルに変換し、softMax関数で確率化。
4. 誤差逆伝搬:正解ラベルと確率の誤差が小さくなるように重みDinを更新。


## 実装例
---
実際にDoc2vecを用いて、分散表現を作成する。 文章はTF-IDfの例と同様に[ferret](https://ferret-plus.com/)の記事5件とその関連記事1件を使用する。 まず必要なファイルをダウンロードした後にmecabを使い記事の形態素解析を行う。

In [ ]:
# mecabに必要なライブラリ取得
!apt-get install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3
!ln -s /etc/mecabrc /usr/local/etc/mecabrc # シンボリックリンクの作成

In [ ]:
# テキストの取得
!wget https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/documents.txt
!wget https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/titles.txt

実装にはgensimに搭載されているDoc2VecrとTaggedDocumentを用いる。 これらを使うことで上に挙げた演算を容易に実装することができる。
DOC2Vecのパラメータについては[doc2vecパラメータ](https://www.gmo-jisedai.com/wp-content/uploads/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2019-02-13-14.41.48.png)に記載

In [ ]:
import numpy as np
import MeCab
import sys
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from collections import OrderedDict

# input_text = open('./titles.txt', 'r').read()
input_text = open('./documents.txt', 'r').read()
documents = input_text.split("|")

def words(text):
    """
        文章から単語を抽出
    """
    out_words = []
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)

    while node:
        word_type = node.feature.split(",")[0]
        if word_type in ["名詞"]:
            out_words.append(node.surface)
        node = node.next
    return out_words



# 学習データとなる各文書
training_docs = []
for i, document in enumerate(documents):
    training_docs.append(TaggedDocument(words=words(document), tags=['doc' + str(i + 1)]))

# min_count=1:最低1回出現した単語を学習に使用
# dm: 学習モデル=0,DBOW 1,DM
model = Doc2Vec(training_docs)

tags = OrderedDict() #辞書の繰り返し時による順番を保つ
tag_list = (('doc1', "記事A"), ('doc2', "記事B"), ('doc3', "記事C"), ('doc4', "記事D"), ('doc5', "記事E"), ('doc6', "記事F"))
dic = OrderedDict(tag_list)
tags.update(dic)

for k, v in tags.items():
    print("[" + v + "]")
    for items in model.docvecs.most_similar(k):
        print("\t" + tags[items[0]] + " : "+ str(items[1]))